In [2]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import time
import glob
from lxml.html import parse
from sphfile import SPHFile
import pydub
import audiosegment
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
import sys

In [3]:
start_df = pd.read_csv('trainfiles/start.csv')
end_df = pd.read_csv('trainfiles/end.csv')

In [4]:
segments_df = start_df.append(end_df)
speakers = Counter(segments_df['speaker'])
common_speakers = speakers.most_common(20)
segments_by_speakers = {speaker:segments_df[segments_df['speaker']==speaker] for speaker, count in common_speakers}
segments_by_speakers
common_speakers = [common_speaker[0] for common_speaker in common_speakers]

## Maybe try increasing dataset size by doing each combination of 290*290

In [5]:
overlay = []
for first_speaker in common_speakers:
    second_speakers = [speaker for speaker in common_speakers if speaker!=first_speaker ]
    first_segments = segments_by_speakers[first_speaker]
    for second_speaker in second_speakers:
        second_segments = segments_by_speakers[second_speaker]
        first_idx = np.arange(290)
        second_idx = np.arange(290)
        np.random.shuffle(first_idx)
        np.random.shuffle(second_idx)
        for i in range(290):
            first_segment = first_segments.iloc[first_idx[i]]
            second_segment = second_segments.iloc[second_idx[i]]
            row = {'first_speaker': first_speaker, 'second_speaker': second_speaker,
                  'first_file': first_segment['segfile'], 'second_file': second_segment['segfile']}
            overlay.append(row)
overlay = pd.DataFrame(overlay)

In [12]:
overlay_shuffled = overlay.sample(frac=1, random_state = 12345)
tenth = int(len(overlay)*0.1)
overlay_shuffled[:8*tenth].to_csv('overlay-train.csv')
overlay_shuffled[8*tenth:9*tenth].to_csv('overlay-val.csv')
overlay_shuffled[9*tenth:].to_csv('overlay-test.csv')
overlay_shuffled

,first_speaker,second_speaker,first_file,second_file
60350,eddie_mair,andrea_arsenault,trainfiles/start_segments/eh971016_seg104.npy,trainfiles/end_segments/d960530a_seg0.npy
107186,andrea_arsenault,brian_lamb,trainfiles/end_segments/d960604a_seg75.npy,trainfiles/start_segments/i960531__seg446.npy
18671,lou_waters,brian_lamb,trainfiles/start_segments/ee970703_seg41.npy,trainfiles/start_segments/ew970627_seg297.npy
36889,ted_koppel,mark_mullen,trainfiles/start_segments/a960612__seg77.npy,trainfiles/start_segments/b960606__seg84.npy
99060,thalia_assuras,andrea_arsenault,trainfiles/start_segments/b960618__seg34.npy,trainfiles/start_segments/d960604a_seg23.npy
...,...,...,...,...
109993,andrea_arsenault,robert_siegel,trainfiles/end_segments/d960515__seg245.npy,trainfiles/start_segments/j960617__seg303.npy
85412,lynn_vaughan,lisa_mullins,trainfiles/end_segments/f960607b_seg50.npy,trainfiles/start_segments/eh971015_seg58.npy
2177,peter_jennings,brian_lamb,trainfiles/start_segments/ea980119_seg56.npy,trainfiles/start_segments/i960610__seg188.npy
77285,mark_mullen,peter_jennings,trainfiles/start_segments/b960613a_seg15.npy,trainfiles/start_segments/c960528__seg21.npy
